In [4]:
import pandas as pd
from pathlib import Path
import time


experiments = [
    {"name": "r18_lr1e-3_fp16", "arch": resnet18, "lr": 1e-3},
    {"name": "r18_lr3e-4_fp16", "arch": resnet18, "lr": 3e-4},
    {"name": "r34_lr1e-3_fp16", "arch": resnet34, "lr": 1e-3},
    {"name": "r34_lr3e-4_fp16", "arch": resnet34, "lr": 3e-4},
    {"name": "r50_lr1e-3_fp16", "arch": resnet50, "lr": 1e-3},
    {"name": "r50_lr3e-4_fp16", "arch": resnet50, "lr": 3e-4},
]


NameError: name 'resnet18' is not defined

In [2]:
models_dir = Path("models")
models_dir.mkdir(exist_ok=True)

all_results = []


In [3]:
for exp in experiments:
    print(f"\n\n==============================")
    print(f"EXPERIMENT: {exp['name']}")
    print(f"==============================")

    exp_start_time = time.time()
    fold_metrics = []

    for fold, (train_idx, valid_idx) in enumerate(
        skf.split(hf_train['img'], hf_train['label'])
    ):
        if (exp["name"], fold) in done_keys:
            print(f"Пропуск: {exp['name']} fold {fold} уже посчитан")
            continue

        print(f"\n==== Fold {fold+1} ====")

        train_ds = Subset(hf_train, train_idx)
        valid_ds = Subset(hf_train, valid_idx)

        train_dl = TfmdDL(
            HFtoFAI(train_ds), bs=128, shuffle=True,
            after_item=item_tfms, after_batch=batch_tfms,
            num_workers=1, device='cpu',
        )

        valid_dl = TfmdDL(
            HFtoFAI(valid_ds), bs=128, shuffle=False,
            after_item=item_tfms, after_batch=valid_batch_tfms,
            num_workers=1, device='cpu',
        )

        dls = DataLoaders(train_dl, valid_dl)
        dls.c = 10
        dls.vocab = list(range(10))

        learn = vision_learner(
            dls,
            exp["arch"],
            pretrained=False,
            n_out=10,
            loss_func=CrossEntropyLossFlat(),
            metrics=[accuracy, error_rate],
            cbs=[
                EarlyStoppingCallback(patience=3),
                SaveModelCallback(
                    monitor='valid_loss',
                    fname=models_dir / f"{exp['name']}_fold{fold}"
                )
            ],
        ).to_fp16()

        fold_start_time = time.time()

        learn.fit_one_cycle(30, exp["lr"])

        fold_time = time.time() - fold_start_time
        val_loss, val_acc, val_err = learn.validate()

        result_row = {
            "experiment": exp["name"],
            "arch": exp["arch"].__name__,
            "lr": exp["lr"],
            "fold": fold,
            "val_loss": float(val_loss),
            "acc": float(val_acc),
            "err": float(val_err),
            "fold_time_sec": round(fold_time, 2),
            "model_path": str(models_dir / f"{exp['name']}_fold{fold}.pth")
        }

        fold_metrics.append(result_row)
        all_results.append(result_row)

        pd.DataFrame(all_results).to_csv(RESULTS_PATH, index=False)

    exp_time_sec = time.time() - exp_start_time

    if fold_metrics:
        best_fold = max(fold_metrics, key=lambda x: x["acc"])
        mean_acc = np.mean([m["acc"] for m in fold_metrics])
        std_acc  = np.std([m["acc"] for m in fold_metrics])

        print(f"\nEXP DONE: {exp['name']}")
        print(f"BEST FOLD: {best_fold['fold']} | ACC={best_fold['acc']:.4f}")
        print(f"MEAN ACC: {mean_acc:.4f} | STD: {std_acc:.4f}")
        print(f"TIME: {exp_time_sec/60:.2f} min")

        for m in fold_metrics:
            m["mean_acc"] = mean_acc
            m["std_acc"]  = std_acc
            m["exp_time_sec"] = round(exp_time_sec, 2)

        pd.DataFrame(all_results).to_csv(RESULTS_PATH, index=False)


NameError: name 'experiments' is not defined